In [24]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import random
from PIL import Image 
import os
import sklearn.metrics as skm

### 獲取要隱藏的訊息

1. 逐字讀取原始訊息
2. 轉換為ascii碼
3. ascii碼轉換為二進制(0bxxxxxxx)的字串
4. replace()替換0b為空
5. 七位數二進制填充為八位數

In [25]:
def getMessage(txtPath):
    byteNumber=0
    with open(txtPath,"rb")  as file:
        txt = file.read()
        byteNumber=len(txt)
        charMessage=""
        for index in range(len(txt)):
            charMessage=charMessage+""+str(bin(txt[index]).replace('0b','')).zfill(7)
            if(index<10):
                print('第'+str(index)+'位為:  '+chr(txt[index])+', 二進制為: '+str(bin(txt[index]).replace('0b','')).zfill(7))
            else:
                print('第'+str(index)+'位為: '+chr(txt[index])+', 二進制為: '+str(bin(txt[index]).replace('0b','')).zfill(7))
    print("輸出為："+charMessage) 
    print("字元數："+str(byteNumber)) 
    print("位元數："+str(len(charMessage))) 
    return charMessage

In [26]:
message=getMessage("../message/1024message.txt")

第0位為:  N, 二進制為: 1001110
第1位為:  I, 二進制為: 1001001
第2位為:  U, 二進制為: 1010101
第3位為:  1, 二進制為: 0110001
第4位為:  1, 二進制為: 0110001
第5位為:  1, 二進制為: 0110001
第6位為:  2, 二進制為: 0110010
第7位為:  C, 二進制為: 1000011
第8位為:  r, 二進制為: 1110010
第9位為:  y, 二進制為: 1111001
第10位為: p, 二進制為: 1110000
第11位為: t, 二進制為: 1110100
第12位為: o, 二進制為: 1101111
第13位為: g, 二進制為: 1100111
第14位為: r, 二進制為: 1110010
第15位為: a, 二進制為: 1100001
第16位為: p, 二進制為: 1110000
第17位為: h, 二進制為: 1101000
第18位為: y, 二進制為: 1111001
第19位為: N, 二進制為: 1001110
第20位為: I, 二進制為: 1001001
第21位為: U, 二進制為: 1010101
第22位為: 1, 二進制為: 0110001
第23位為: 1, 二進制為: 0110001
第24位為: 1, 二進制為: 0110001
第25位為: 2, 二進制為: 0110010
第26位為: C, 二進制為: 1000011
第27位為: r, 二進制為: 1110010
第28位為: y, 二進制為: 1111001
第29位為: p, 二進制為: 1110000
第30位為: t, 二進制為: 1110100
第31位為: o, 二進制為: 1101111
第32位為: g, 二進制為: 1100111
第33位為: r, 二進制為: 1110010
第34位為: a, 二進制為: 1100001
第35位為: p, 二進制為: 1110000
第36位為: h, 二進制為: 1101000
第37位為: y, 二進制為: 1111001
第38位為: N, 二進制為: 1001110
第39位為: I, 二進制為: 1001001
第40位為: U, 二進制為: 1010101
第41位為: 1, 二進制為: 

In [27]:
#取得單一字元二元序列
def getACharMessage(message,index):
    return message[(index*7):(index*7)+7]

### 取得原始影像資料

1. 將影像轉成灰階
2. 轉換成二維像素陣列

In [28]:
#取得原始圖像像素陣列與大小
def getImageArray(dirName):
    imageArray=[]
    for imageName in os.listdir(dirName):
        imageArray.append(cv2.imread(dirName+'/'+imageName))
    return imageArray,len(imageArray)

In [29]:
#取得修剪後圖像矩陣陣列與大小
def getResizeImageArray():
    imageResizeArray=[]
    imageArray,imageArrayLength=getImageArray('../database/resizeImage')
    for image in imageArray:
        imageResizeArray.append(cv2.cvtColor(image,cv2.COLOR_BGR2GRAY))
    return imageResizeArray,imageArrayLength

In [50]:
# 多個圖像像素陣列
imageArray,imageArrayLength=getResizeImageArray()
# 測試圖像
testImage=Image.open('../images/lennaGrayScaleCrop.jpg') 

## 將訊息隱藏進去

1. 分別將每個像素點的RGB值餘2，去掉最低位的值
2. 再從需要隱藏的信息中取出一位，轉換為整型
3. 兩值相加，就把信息隱藏起來了

In [52]:
# 取餘數
def mod(x,y):
    return x%y

In [58]:
def hideMessagePixel(testImage,message,afterSrc):
    #獲取圖片的寬和高
    width,height= testImage.size[0],testImage.size[1]
    print("width:"+str(width))
    print("height:"+str(height))

    count = 0

    #獲取需要隱藏的信息 
    messageLength = len(message)
    for h in range(height):
        for w in range(width):
            pixel = testImage.getpixel((w,h))
            a=pixel[0]
            b=pixel[1]
            c=pixel[2]
            
            if count == messageLength:
                break
            
            a= a-mod(a,2)+int(message[count])
            print('Pixel('+str(w)+','+str(h)+')= '+str(pixel),'AfterHiddedPixel= '+str((a,b-mod(b,2)+int(message[count]),c-mod(c,2)+int(message[count]))))

            count+=1
            if count == messageLength:
                testImage.putpixel((w,h),(a,b,c)) 
                break
            b =b-mod(b,2)+int(message[count])
            count+=1 
            if count == messageLength:
                testImage.putpixel((w,h),(a,b,c)) 
                break
            c= c-mod(c,2)+int(message[count])
            count+=1
            if count == messageLength:
                testImage.putpixel((w,h),(a,b,c))
                break
            if count % 3 == 0:
                testImage.putpixel((w,h),(a,b,c))
    testImage.save(afterSrc)

In [59]:
hideMessagePixel(testImage,message,'../images/LennaAfterLSB.jpg')

width:640
height:480
Pixel(0,0)= (255, 255, 255) AfterHiddedPixel= (255, 255, 255)
Pixel(1,0)= (255, 255, 255) AfterHiddedPixel= (255, 255, 255)
Pixel(2,0)= (255, 255, 255) AfterHiddedPixel= (254, 254, 254)
Pixel(3,0)= (255, 255, 255) AfterHiddedPixel= (254, 254, 254)
Pixel(4,0)= (255, 255, 255) AfterHiddedPixel= (254, 254, 254)
Pixel(5,0)= (255, 255, 255) AfterHiddedPixel= (254, 254, 254)
Pixel(6,0)= (255, 255, 255) AfterHiddedPixel= (255, 255, 255)
Pixel(7,0)= (255, 255, 255) AfterHiddedPixel= (254, 254, 254)
Pixel(8,0)= (255, 255, 255) AfterHiddedPixel= (254, 254, 254)
Pixel(9,0)= (255, 255, 255) AfterHiddedPixel= (255, 255, 255)
Pixel(10,0)= (255, 255, 255) AfterHiddedPixel= (255, 255, 255)
Pixel(11,0)= (255, 255, 255) AfterHiddedPixel= (254, 254, 254)
Pixel(12,0)= (255, 255, 255) AfterHiddedPixel= (255, 255, 255)
Pixel(13,0)= (255, 255, 255) AfterHiddedPixel= (254, 254, 254)
Pixel(14,0)= (255, 255, 255) AfterHiddedPixel= (254, 254, 254)
Pixel(15,0)= (255, 255, 255) AfterHiddedPixe

### PSNR
峰值訊噪比是一個表示訊號最大可能功率和影響它的表示精度的破壞性雜訊功率的比值的工程術語

In [62]:
# 計算 psnr
def psnr(originImage,hidedImage):
   diffrence = originImage-hidedImage
   mse = np.mean(np.square(diffrence))
   psnr = 10 * np.log10(255 * 255 / mse)
   return(psnr)

In [60]:
LSBImage = cv2.imread('../images/LennaAfterLSB.jpg')
LSBImage = cv2.cvtColor(LSBImage,cv2.COLOR_BGR2GRAY) 
grayscaleImage = cv2.imread('../images/lennaGrayScaleCrop.jpg')
grayscaleImage = cv2.cvtColor(grayscaleImage,cv2.COLOR_BGR2GRAY) 

In [63]:
print('PSNR: '+str(psnr(grayscaleImage,LSBImage)))

PSNR: 66.0545791340609


### 相對熵 
又稱 KL散度（Kullback-Leibler divergence，簡稱KLD）

reference: https://blog.csdn.net/sihaiyinan/article/details/112196356

In [64]:
def hxx_forward(x, y):
    return skm.mutual_info_score(x, y)

In [65]:
def hxx(x, y):
    size = x.shape[-1]
    px = np.histogram(x, 256, (0, 255))[0] / size
    py = np.histogram(y, 256, (0, 255))[0] / size
    hx = - np.sum(px * np.log(px + 1e-8))
    hy = - np.sum(py * np.log(py + 1e-8))
 
    hxy = np.histogram2d(x, y, 256, [[0, 255], [0, 255]])[0]
    hxy /= (1.0 * size)
    hxy = - np.sum(hxy * np.log(hxy + 1e-8))
 
    r = hx + hy - hxy
    return r

In [66]:
x = np.reshape(grayscaleImage, -1)
y = np.reshape(LSBImage, -1)
 
print("相對熵= "+str(hxx_forward(x, y)))

相對熵= 3.1133722085738853
